In [11]:
# @desc   The summary dashboard for the building permit project
#         This file gets the outcomes from the building-permits.ipynb
# @author Magic Magg
# @date   Feb 13, 2024

import plotly.io as pio
pio.renderers.default='notebook'

# the budget of the clients in US dollars
BUDGET=400000
MIN_BUDGET=2000

# where the baseline cost file should be stored (the file is generated by
# building-permits.ipynb)
DATA_PATH_COST='input/Building_Permits_20231024_cost.csv.zip'


import pandas as pd
import numpy as np
import plotly.graph_objects as go

# my routines
import myutils as ut



# get our cost data frame; it uses a file with costs generated by building-permits.ipynb
df = ut.read_to_df(DATA_PATH_COST, False)

# the minimal date to be considered by the clients
MIN_DATE = pd.to_datetime('2022-01-01').date()
MAX_DATE = pd.to_datetime('2023-10-24').date()

# updates should be done 
'''
    get min or max date from a frame
    @param d_ a dataframe
    @return max or min value
'''
get_extreme = lambda d, is_max=True : d['AppliedDate'].max().date() if is_max else d['AppliedDate'].min().date()

# get the dates min and max dates
MIN_DATE = get_extreme(df, is_max=False)
MAX_DATE = get_extreme(df)

# the style of the map
MAP_STYLE = "open-street-map"
#MAP_STYLE = "stamen-terrain"

# -------------------------
# Dash app
# -------------------------

from dash import Dash, html, dash_table, dcc, callback, Output, Input, State

import plotly.express as px

# costs and projects your money can buy
projects_md_text = """
## Examples of Construction Projects Given a Specific Budget

### < $10K

- Deck addition/replacement
- Constructing a garage

### < $50K

- Constructing a detached home accessory
- Converting a garage to a heated office

### < $100K

- Adding a new dwelling accessory
- Installing a new one-family dwelling
- Constructing one-story addition

### > $100K

- $100K+ projects very often involved constructing AADU or DADU (attached/detatched
  dwelling units)

### > $200K

- Constructing one or more family dwelling unit.

"""

permits_md_text = """ 
## Permit Expiration

The permit expires typically after **548** days from the issue date regardless of the
permit type.

"""


about_md_text = """ 

## Purpose

The dashboard summarizes findings regarding building permits issued by
Seattle Department of Construction and Inspections with respect to clients'
requirements.

The clients want to remodel their house that might include house modification,
or demolition. They want to gather tentative data regarding the cost and 
total project duration.

## Business Tasks

The purpose is to present exploratory analysis results and answer the following
questions:

1. How much does remodeling of the house cost?
2. How long does it take to complete the house remodeling project?
3. What is geography of the remodeling projects with respect to cost in Seattle?

## Constraints

The clients have posed the following constraints:

- Budget <= $400,000
- Timeframe: Jan 1, 2022-Oct 24, 2023.

## Data Source

The analysis is based on public records provided by
[Seattle Department of Construction and Inspections, 2023.](https://data.seattle.gov/Permitting/Building-Permits/76t5-zqzr)

"""

# preparing dat for the Estimated Project Cost
prices_df = df.copy()
# the prefix for the building PermitTypeMapped
BUILD_PFX = ''
# the value for the demolition/deconstructions
DEMO_PFX = 'Demolition'

# names for checkboxes
BUILD_OPT = 'Building'
DEMO_OPT = 'Demolition'

# I will remove some columns that do not seem necessary
prices_df = prices_df.loc[:, ~prices_df.columns.isin(['PermitClass', 'PermitClassMapped', 
                                          'RelatedMup', 'Location1', 
                                          'CostBinned'])]

#print(f"Count of None PermitTypeMapped = {len(prices_df[prices_df['PermitTypeMapped'].isna()])}")
#print(f"Unique vals for PermitTypeMapped={prices_df['PermitTypeMapped'].unique()}")
# I compact all Demolitions into one category because there are not that much
# (please see building-permits.ipynb for details)
# so results will be a little bit different from the building-permits.ipynb
# there are None values that result in nan about 46 nan values in the new column
# for Demolition so I do not concat it with the PermitTypeDesc col
prices_df['PermitTypeDesc2'] = np.where(prices_df['PermitTypeMapped'] == 'Building', BUILD_PFX + prices_df['PermitTypeDesc'], DEMO_PFX)
#print(prices_df['PermitTypeDesc2'].unique())
#print(prices_df[prices_df['PermitTypeDesc2'].isna()])
#permit_counts = prices_df.groupby('PermitTypeDesc2').PermitTypeDesc2.value_counts()
#print(permit_counts.index)

# I will add the cost binning for the  prices
labels = ['(0, 10K]', '(10K, 50K]', '(50K, 100K]', '(100K, 200K]', '(200K, 300K]', '(300K, 400K]']
prices_df['CostBinned2'] = pd.cut(
        prices_df['EstProjectCost'],
        [0, 1.0e+04, 5.0e+04, 1.0e+05, 2.0e+05, 3.0e+05, BUDGET],
        BUDGET,
        labels = labels,
        include_lowest=True,
        ordered = True
    )
# print(prices_df['CostBinned2'].unique())
# print(prices_df.info())


# separate for buildings and demolition projects
build_df = prices_df[prices_df['PermitTypeMapped'] == 'Building'].copy()
demo_df = prices_df[prices_df['PermitTypeMapped'] == 'Demolition'].copy()

# print(f"Building + Demolition = {len(build_df.index) + len(demo_df.index)}, Building = {len(build_df.index)}, All = {len(prices_df.index)}")

# for showing the duration results, i.e., how long project went
times_df = prices_df[ prices_df['AppliedDate'].notna() 
            & prices_df['IssuedDate'].notna() 
            & prices_df['ExpiresDate'].notna() 
            & prices_df['CompletedDate'].notna()].copy()
# print(f"times_df rec count={len(times_df.index)}")

get_days_count = lambda f, minuend_col, subtrahend_col : (f[minuend_col] - f[subtrahend_col]).dt.days

times_df['ApplToComplDays'] = get_days_count(times_df, 'CompletedDate', 'AppliedDate')
times_df['ApplToIssueDays'] = get_days_count(times_df, 'IssuedDate', 'AppliedDate')
times_df['IssuedToComplDays'] = get_days_count(times_df,'CompletedDate', 'IssuedDate')

appl_compl_series = times_df.groupby('PermitTypeDesc2').ApplToComplDays.median()
appl_issued_series = times_df.groupby('PermitTypeDesc2').ApplToIssueDays.median()
issued_compl_series = times_df.groupby('PermitTypeDesc2').IssuedToComplDays.median()

duration_df = pd.concat([appl_compl_series, appl_issued_series, issued_compl_series], axis=1)

#print(f"duration_df={duration_df}")
duration_df = np.ceil(duration_df)
duration_df = duration_df.astype(int)
duration_df.reset_index(inplace=True)
#print(f"duration_df={duration_df}")

get_cols= lambda f, col_name: f[['PermitTypeDesc2', col_name]].sort_values(col_name, ascending=False).to_markdown(index=False)

"""
    Produce a string describing counts per permit group
    Helpful for generating the info for the plot titles
    @param df_ dataframe
    @return string describing counts per permit group
"""
def count_grouped_permits(df_):
    permit_counts = df_.groupby('PermitTypeDesc2').PermitTypeDesc2.value_counts()
    # format a string from a series for the title
    s = ''
    for x in permit_counts.index:
        s = s + x + '=' + str(permit_counts[x]) + ', '
    # remove to last characters
    s = s[:-2]

    return s

# this produces a string of how many records in each permit group
# compute now, that number does not change use for titles
times_df_rec_str = count_grouped_permits(times_df)


medians_md_text1 = f"""
## Total Project Duration: From Application To Completion (Medians):
"""
# add it to markdown if you want to see the table
#{get_cols(duration_df, 'ApplToComplDays')}


medians_md_text2 = f"""
## From Application To Permit Issuance (Medians):
"""
#{get_cols(duration_df, 'ApplToIssueDays')}

medians_md_text3 = f"""
## From Permit Issued To Project Completion (Medians):
"""
#{get_cols(duration_df, 'IssuedToComplDays')}


"""
    plots the estimated project costs in boxplots
    @param df_ the dataframe
    
    @return fig
"""
def plotbox_prices(df_):
    
    # the total number of all records
    rec_count = len(df_.index)

    permit_counts = df_.groupby('PermitTypeDesc2').PermitTypeDesc2.value_counts()
    # format a string from a series for the title
    s = ''
    for x in permit_counts.index:
        s = s + x + '=' + str(permit_counts[x]) + ', '
    # remove to last characters
    s = s[:-2]
    
    fig = px.box(df_.sort_values('PermitTypeDesc2'), 
                 x='PermitTypeDesc2', y = 'EstProjectCost', points='all',
                 title=f"Estimated Project Cost. Total number of records: {rec_count};<BR>{s}.")
    return fig

"""
    Plot the mapbox

    @param df_ (Dataframe) the dataframe
    @param markers_visible (boolean) if True they are visible, otherwise not
    @return 
"""
def plot_mapbox(df_, markers_visible=True):
    
    title = f"Estimated Project Cost <= ${BUDGET} for Building Permits Issued By<BR>Seattle Department Of Construction and Inspections from {MIN_DATE} to {MAX_DATE}."
    
    # holder for the figure
    fig = None

    if (df_ is None) or len(df_.index) == 0:
        # Plotly Express cannot create empty figures, so the examples below mostly create an "empty"
        fig = go.Figure(go.Scattergeo( ))
        fig.update_geos(scope='usa',
                        title = f"ERROR: The dataframe is empty.<BR>{title}")
    else:
        fig = px.scatter_mapbox(df_, lat="Latitude", lon="Longitude", 
                            title = title, 
                            color="CostBinned2", 
                            mapbox_style=MAP_STYLE,
                            category_orders= {'CostBinned2' : labels },
                            hover_name = 'CostBinned2',
                            # 'Description' sometimes is too long
                            hover_data = ['EstProjectCost','PermitNum', 'PermitTypeMapped', 'PermitTypeDesc'],
                            #color_continuous_scale= px.colors.sequential.Bluered,
                            #color_continuous_scale= px.colors.sequential.Turbo,
                            color_discrete_sequence= px.colors.sequential.Inferno_r,
                            #px.colors.cyclical.IceFire, 
                            size_max=15, zoom=10,
                            width = 800, height = 900)
        # correct the figure for markers, this is a hack because the mapbox
        # can't render the empty dataframe, so I make the markers invisible
        # if everything will be unchecked
        if not markers_visible:
            # make the markers invisible;
            fig.update_traces(
                marker=go.scattermapbox.Marker(opacity=0.0))
            fig.update_layout(hovermode=False, showlegend=False)
    
    return fig




"""
    @param df_ a dataframe
    @param col_name_ A name for a difference column
    @param minuend_col_ A name of the column that we subtract from
    @param subtrahend_col_ A name of the column that we subtract
    @param rec_count_str_ The info of how many records in each permit group (used
           for title)

    @return fig
"""
def plot_timebox(df_, col_name_, minuend_col_, subtrahend_col_, rec_count_str_):    
    
    fig = px.box(df_, x='PermitTypeDesc2', y = col_name_, points='all',
                 title=f"Duration of the project in days from {subtrahend_col_} till {minuend_col_}.<BR>"
                       f"Number of records = {len(df_.index)}: {rec_count_str_}.")
    fig.update_layout(yaxis_title=f"Duration = {minuend_col_} - {subtrahend_col_} [Days]")
    
    return fig

"""
    Plot the medians of dataframe to compare which projects complete faster.
    @param df_ medians dataframe
    @param rec_count_str_ The info of how many records in each permit group

    
    @return  fig
"""
def plot_medians(df_, rec_count_str_):

    f = df_.sort_values(['IssuedToComplDays'],ascending=[True])
    s = count_grouped_permits(f)
    

    fig = go.Figure()
    fig.add_trace(go.Bar(
        y = f['PermitTypeDesc2'],
        x = f['IssuedToComplDays'],
        name = 'Doing Project',
        orientation = 'h',
        #marker=dict(
        #    color='rgba(246, 78, 139, 0.6)',
        #    line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
        #)
    ))
    fig.add_trace(go.Bar(
        y = f['PermitTypeDesc2'],
        x = f['ApplToIssueDays'],
        name = 'Waiting For Permit',
        orientation = 'h',
        #marker=dict(
        #   color='rgba(58, 71, 80, 0.6)',
        #    line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
        #)
    ))
    
    fig.update_layout(barmode='stack', 
                      title=f'Execute the permit vs. Wait for the permit (medians): {rec_count_str_}')    

    return fig

"""
    Plots a bar chart
    @param df_ dataframe
    @param rec_count_str_ The info of how many records in each permit group

    @return 
"""
def plot_bar(df_, rec_count_str_):
    cols = df_.columns
    f = df_.sort_values(cols[1], ascending=False)
    fig = px.bar(f, x=cols[1], y=cols[0], orientation='h', 
                 title=f"Medians of days required for {cols[1]}. Records count: {rec_count_str_}")

    return fig


# init the app, Dash constructor
# we suppress callback exceptions because we have input and output id
# generated in callbacks
app = Dash(__name__, suppress_callback_exceptions=True)

# the app components that will be displayed in the web browser
app.layout = html.Div([
    html.H1(children=f'Building Permits <= ${BUDGET} from {MIN_DATE} to {MAX_DATE} based on public records from Seattle Department of Construction and Inspections', style={'textAlign':'center'}),
    
    dcc.Tabs(id='id_tabs', value='tab-map', children=[
        dcc.Tab(label='Map', value='tab-map'),
        dcc.Tab(label='Cost', value='tab-cost'),
        dcc.Tab(label='Time', value = 'tab-permits'),
        dcc.Tab(label='Records', value='tab-records'),
        dcc.Tab(label='About', value = 'tab-about')
    ]),

    html.Div(id='id_tabs_content')
])

# the tab for map
tab_map = html.Div([
            html.H3('Select Permit Type'),

            dcc.Checklist(options=[BUILD_OPT, DEMO_OPT],  
                          id='id_permit_type', 
                          #value = [],
                          value = [BUILD_OPT, DEMO_OPT],
                          inline=True),
            #dcc.Graph(figure=plot_mapbox(empty_df), id='id_map_graph'),
            dcc.Graph('id_map_graph')
        ])


# decorator for the render_content function that will take 
# care of rendering the tabs
@callback(
    Output(component_id='id_tabs_content', component_property='children'),
    Input(component_id='id_tabs', component_property='value')
)
def render_content(tab):
    if tab == 'tab-map':        
        return tab_map
    elif tab == 'tab-cost':
        return html.Div([            
            dcc.Markdown(children=projects_md_text),
            dcc.Graph(figure=plotbox_prices(prices_df), id='id_cost_graph'),           
        ])
    elif tab == 'tab-permits':
        return html.Div([
            dcc.Markdown(permits_md_text),
            dcc.Markdown("## Medians Of The Project Execution And Waiting Time (In Days):"),
            dcc.Graph(figure=plot_medians(duration_df, times_df_rec_str), id='id_medians_graph'),
            dcc.Markdown(medians_md_text1),
            dcc.Graph(figure = plot_bar(duration_df[['PermitTypeDesc2', 'ApplToComplDays']], times_df_rec_str)),
            dcc.Graph(figure=plot_timebox(times_df, 'ApplToComplDays', 'CompletedDate', 'AppliedDate', times_df_rec_str), id='id_proj_duration_graph'),                        
            dcc.Markdown(medians_md_text2),
            dcc.Graph(figure = plot_bar(duration_df[['PermitTypeDesc2', 'ApplToIssueDays']], times_df_rec_str)),            
            dcc.Graph(figure=plot_timebox(times_df, 'ApplToIssueDays', 'IssuedDate', 'AppliedDate', times_df_rec_str), id='id_issue_graph'),                        
            dcc.Markdown(medians_md_text3),
            dcc.Graph(figure = plot_bar(duration_df[['PermitTypeDesc2', 'IssuedToComplDays']], times_df_rec_str)),             
            dcc.Graph(figure=plot_timebox(times_df, 'IssuedToComplDays', 'CompletedDate', 'IssuedDate', times_df_rec_str), id='id_issue_compl_graph')
            
        ])
    elif tab == 'tab-records':
        return html.Div([
            html.Br(),
            dcc.Input(
                id='id_input_box',
                placeholder='Enter the permit number ...' ,
                type = 'text',               
            ),
            html.Button('Find', id='id_find_btn'),
            html.Div(id='id_output_find', style={'whiteSpace': 'pre-line'}),
            html.Br(),
            dash_table.DataTable(data=prices_df.to_dict('records'), 
                                 page_size=10,
                                 sort_action = 'native',
                                 style_table = { 'overflowX' : 'auto'},
                                 id = 'id_table',
                                 columns=[{"name": i, 'id': i} for i in prices_df.columns],
                                 filter_action = 'native',
                                 style_data={
                                 'width': '150px', 'minWidth': '150px', 'maxWidth': '150px',
                                 'overflow': 'hidden',
                                    'textOverflow': 'ellipsis',}
                                )
                                            
        ])
    elif tab == 'tab-about':
        return html.Div([
            dcc.Markdown(about_md_text)
        ])
    else:
        return html.Div([
            html.H3(f'ERROR: tab {tab} not detected...')
        ])


@app.callback(
    Output('id_map_graph', 'figure'),
    Input('id_permit_type', 'value')
)
def update_map(permit_types):
    if len(permit_types) == 2:
        fig = plot_mapbox(prices_df)
    elif len(permit_types) == 1:
        fig = plot_mapbox(build_df) if permit_types[0] == BUILD_OPT else plot_mapbox(demo_df)
    else:
        # TODO it should be just cleaning an existing map from markers but it
        # is just redrawing a map and making markers invisible 
        fig = plot_mapbox(demo_df, False)

    return fig

@app.callback(
    Output('id_output_find', 'children'),
    Input('id_find_btn', 'n_clicks'),
    State('id_input_box', 'value'),
)
def update_find_output(n_clicks, permit_number):
    # I take the information from the dataframe, not from the DataTable
    # For now it works as I want to have information about a record
    f = prices_df[prices_df['PermitNum'] == permit_number]
    if f is None or len(f.index) == 0:
        return f"Permit '{permit_number}' NOT FOUND"
    else:
        
        return (f"Estimated Cost : ${f.iloc[0]['EstProjectCost']}\n"
                f"Description:\n{f.iloc[0]['Description']}\n"
                f"Status: {f.iloc[0]['StatusCurrent']}\n"
                f"Contractor: {f.iloc[0]['ContractorCompanyName']}\n"
                f"Link: {f.iloc[0]['Link']}"
                )    

# run the app
if __name__ == '__main__':
    # this is Dash "hot-reloading"
    # dash will automatically refresh the browser when you make a change in your code
    # it can be turned off with dev_tools_hot_reload=False
    app.run(debug=True)


